# Football Database

In this notebook I analyze a database that contains data from  European Football.

The database is composed for 7 tables which I describe below:

* `Country`: 11 rows and 2 columns

    - id: Country id
    - name: Name of the country

* `League`: 11 rows and 3 columns

    - id: League id
    - country_id: Country id of the League
    - name: Name of the League

* `Match`: 26k rows and 115 columns

    - id: Id of the match
    - country_id: Id of the country
    - league_id: Id of the league
    - season: Season the match happened in (goes from 2008/2009 to 2015/2016 season)
    - home_team_goal
    - others
    
* `Player`: 11.1k rows and 7 columns

    - id: Id of the player
    - player_name: Name of the player
    - birthday: Date of birth of player
    - height: Height of the player
    - weight: Weight of the player
    - others
    
* `Player_Attributes`: 184k rows and 42 columns:
 
    - overall_rating
    - potential
    - preferred_foot
    - attacking_work_rate
    - defensive_work_rate
    - crossing
    - others
    
* `Team`: 299 rows and 5 columns

    - id: Id of the team
    - team_api_id
    - team_fifa_api_id
    - team_long_name
    - team_short_name

* `Team_Attributes`: 1458 rows and 25 columns

    - id: Id of the team
    - buildUpPlaySpeed
    - buildUpPlaySpeedClass
    - buildUpPlayDribbling
    - buildUpPlayDribblingClass
    - buildUpPlayPassing
    - buildUpPlayPassingClass

This is a very extensive dataset with more than 11000 players, 300 teams and more than 25k matches. Most of the attributes I mention above are the ones I will be using to answer different questions. 

## Goals

The main goal of this analysis if the use of SQL (SQLite) language to extract analytical information to answer specific questions and provide different insights. The topics I address will be based on the following points:

* Best teams per league 
* Best league: I will focus on the 5 biggest and best known leagues: Spain, France, Germany, England and Italy
* Comparison of the teams per league according to different metrics
* Players with best features
* Relationship between attributes and the number of victory. Which type of attributes took the team to the most victories?


In [1]:
# Installing ipython-sql to use SQL with Python
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\manit\anaconda3\envs\sql_analysis

  added / updated specs:
    - ipython-sql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       h5b45459_0         173 KB  conda-forge
    certifi-2020.12.5          |   py39hcbf5309_0         144 KB  conda-forge
    ipython-sql-0.3.9          |py39hde42818_1002          28 KB  conda-forge
    openssl-1.1.1i             |       h8ffe710_0         5.8 MB  conda-forge
    prettytable-2.0.0          |     pyhd8ed1ab_0          22 KB  conda-forge
    python_abi-3.9             |           1_cp39           4 KB  conda-forge
    sqlalchemy-1.3.20          |   py39h4cdbadb_0         1.8 MB  conda-forge
    sqlparse-0.4.1             |     pyh9f0ad1d_0          34 KB  conda-forge
    ------------------------------------------------------

**Connecting the Julyter notebook to the database file: database.sqlite**

In [2]:
%%capture
%load_ext sql
%sql sqlite:///database.sqlite

## **Extracting information from the database**

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///database.sqlite
Done.


[('table', 'sqlite_sequence', 'sqlite_sequence', 4, 'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table', 'Player_Attributes', 'Player_Attributes', 11, 'CREATE TABLE "Player_Attributes" (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`player_fifa_api_id`\tINTEGER,\n\t`player_api_id`\tINTEGER,\n\t`dat ... (1024 characters truncated) ... tFOREIGN KEY(`player_fifa_api_id`) REFERENCES `Player`(`player_fifa_api_id`),\n\tFOREIGN KEY(`player_api_id`) REFERENCES `Player`(`player_api_id`)\n)'),
 ('table', 'Player', 'Player', 14, 'CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`player_api_id`\tINTEGER UNIQUE,\n\t`player_name`\tTEXT,\n\t`player_fifa_api_id`\tINTEGER UNIQUE,\n\t`birthday`\tTEXT,\n\t`height`\tINTEGER,\n\t`weight`\tINTEGER\n)'),
 ('table', 'Match', 'Match', 18, 'CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`country_id`\tINTEGER,\n\t`league_id`\tINTEGER,\n\t`season`\tTEXT,\n\t`stage`\t ... (4533 characters truncated) ... id`),\n\tFOREIGN KEY(`away_player_10`) REFERENCES `Player`(`player_api_id`),\n\tFOREIGN KEY(`away_player_11`) REFERENCES `Player`(`player_api_id`)\n)'),
 ('table', 'League', 'League', 24, 'CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`country_id`\tINTEGER,\n\t`name`\tTEXT UNIQUE,\n\tFOREIGN KEY(`country_id`) REFERENCES `country`(`id`)\n)'),
 ('table', 'Country', 'Country', 26, 'CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`name`\tTEXT UNIQUE\n)'),
 ('table', 'Team', 'Team', 29, 'CREATE TABLE "Team" (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`team_api_id`\tINTEGER UNIQUE,\n\t`team_fifa_api_id`\tINTEGER,\n\t`team_long_name`\tTEXT,\n\t`team_short_name`\tTEXT\n)'),
 ('table', 'Team_Attributes', 'Team_Attributes', 2, 'CREATE TABLE `Team_Attributes` (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`team_fifa_api_id`\tINTEGER,\n\t`team_api_id`\tINTEGER,\n\t`date`\tTE ... (754 characters truncated) ... s`\tTEXT,\n\tFOREIGN KEY(`team_fifa_api_id`) REFERENCES `Team`(`team_fifa_api_id`),\n\tFOREIGN KEY(`team_api_id`) REFERENCES `Team`(`team_api_id`)\n)')]

**Extracting name of the tables**

In [3]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ("table", "view")

 * sqlite:///database.sqlite
Done.


name,type
sqlite_sequence,table
Player_Attributes,table
Player,table
Match,table
League,table
Country,table
Team,table
Team_Attributes,table
best_leagues,view


## Extracting information of our teams and leagues

**Name of the leagues**

In [4]:
%%sql
SELECT * FROM league;

 * sqlite:///database.sqlite
Done.


id,country_id,name
1,1,Belgium Jupiler League
1729,1729,England Premier League
4769,4769,France Ligue 1
7809,7809,Germany 1. Bundesliga
10257,10257,Italy Serie A
13274,13274,Netherlands Eredivisie
15722,15722,Poland Ekstraklasa
17642,17642,Portugal Liga ZON Sagres
19694,19694,Scotland Premier League
21518,21518,Spain LIGA BBVA


**Note:** As I mentioned above, I will focus on the 5 best known leagues: 

    * England Premier League
    * France Ligue 1
    * Italy Serie A
    * Germany 1. Bundesliga
    * Spain LIGA BBVA
    
I will create a view with the data of only these views

# 1. Wrangling Data

**Creating a VIEW with the best leagues**

In [5]:
%%sql
DROP VIEW IF EXISTS best_leagues;

CREATE VIEW best_leagues AS
SELECT id, country_id,
CASE 
    WHEN name = 'England Premier League' THEN 'Premier League'
    WHEN name = 'France Ligue 1' THEN 'League 1'
    WHEN name = 'Germany 1. Bundesliga' THEN 'Bundesliga'
    WHEN name = 'Italy Serie A' THEN 'Serie A'
    WHEN name = 'Spain LIGA BBVA' THEN 'La Liga'
END AS name
FROM league
WHERE name IN ('England Premier League', 'France Ligue 1', 'Germany 1. Bundesliga', 'Italy Serie A','Spain LIGA BBVA');

SELECT * FROM best_leagues;

 * sqlite:///database.sqlite
Done.
Done.
Done.


id,country_id,name
1729,1729,Premier League
4769,4769,League 1
7809,7809,Bundesliga
10257,10257,Serie A
21518,21518,La Liga


**Creating a VIEW with the teams of the biggest leagues**

In [8]:
%%sql
SELECT t.* FROM team t LIMIT 1;

 * sqlite:///database.sqlite
Done.


id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
1,9987,673,KRC Genk,GEN


In [2]:
%%sql
DROP VIEW IF EXISTS best_teams;

CREATE VIEW best_teams AS
SELECT t.* FROM team t
    INNER JOIN Match m ON m.home_team_api_id = t.team_api_id
    INNER JOIN best_leagues l ON l.id = m.league_id;
    
SELECT COUNT(*) FROM best_teams

 * sqlite:///database.sqlite
Done.
Done.
Done.


COUNT(*)
14585


**Creating a VIEW for the players of the best teams**

I will be analyzing the players that played in the teams that belonged to the best leagues (the main 5 leagues) 

In [11]:
%%sql

SELECT * FROM player LIMIT 0;

 * sqlite:///database.sqlite
Done.


id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight


In [3]:
%%sql
SELECT * FROM match 
WHERE season = '2011/2012' LIMIT 5;

 * sqlite:///database.sqlite
Done.


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
757,1,1,2011/2012,1,2011-07-29 00:00:00,1032692,1773,8635,2,1,1,2,4,6,8,3,5,7,3,5,7,1,3,4,7,1,3,5,7,9,4,6,1,3,3,3,3,7,7,7,10,10,10,1,3,3,3,7,7,7,7,7,10,10,37993,37865,37051,45840,179059,37981,38791,37963,38777,45865,68114,38391,38389,208493,149150,40536,38253,114333,178249,265123,46552,181276,None,None,None,None,None,None,None,None,7,4,1.5,6.5,4.1,1.42,5,3.7,1.5,5.5,3.6,1.5,None,None,None,5.5,3.75,1.57,6,3.6,1.57,7,4,1.5,6,3.75,1.5,6.5,4,1.44
758,1,1,2011/2012,1,2011-07-30 00:00:00,1032693,9998,9985,1,1,1,1,3,5,7,9,3,4,8,6,5,1,2,4,6,8,6,5,4,3,5,7,1,3,3,3,3,3,7,7,7,7,11,1,3,3,3,3,6,8,6,10,10,10,39153,39977,181574,166577,38906,36849,21753,37128,149258,26224,78902,38797,129462,245653,164229,33620,38969,17276,119117,38382,248689,46335,None,None,None,None,None,None,None,None,5,3.5,1.73,4.75,3.5,1.65,3.8,3.3,1.75,4.5,3.5,1.62,None,None,None,4.33,3.5,1.75,4.5,3.5,1.73,4.75,3.6,1.75,4.5,3.4,1.72,4.5,3.6,1.67
759,1,1,2011/2012,1,2011-07-30 00:00:00,1032694,9987,9993,3,1,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,3,5,7,3,5,7,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,10,10,10,91929,94462,38368,148314,109331,104411,39498,169200,43158,42153,38794,36873,57078,38800,174363,27508,38784,163613,38371,33622,166679,14487,None,None,None,None,None,None,None,None,1.44,4.33,7,1.45,3.95,6.25,1.5,3.7,5,1.44,3.75,6,None,None,None,1.4,4.33,7,1.44,4.2,6.5,1.44,4.5,7,1.45,4,6.25,1.44,4,6.5
760,1,1,2011/2012,1,2011-07-30 00:00:00,1032695,9991,9984,0,1,1,1,3,5,7,9,3,5,7,4,6,1,2,4,6,8,3,5,7,3,5,7,1,3,3,3,3,3,7,7,7,10,10,1,3,3,3,3,7,7,7,10,10,10,37854,12473,114368,178096,37440,None,33662,26771,166618,148329,12574,36835,38342,243250,37047,38789,27110,166670,188231,277766,38251,209855,None,None,None,None,None,None,None,None,1.57,3.8,6,1.55,3.85,5,1.55,3.5,4.8,1.57,3.5,5,None,None,None,1.6,3.75,5,1.5,3.9,6,1.53,4,6,1.55,3.75,5.5,1.53,3.75,5.5
761,1,1,2011/2012,1,2011-07-30 00:00:00,1032696,9994,10000,0,0,1,3,4,7,1,3,5,7,9,4,6,1,1,3,5,7,9,3,5,7,4,6,1,3,3,3,7,7,7,7,7,10,10,1,3,3,3,3,3,7,7,7,10,10,30934,94030,25791,166675,95609,38290,67898,30910,42706,104406,104404,37900,37100,41005,46877,80678,37886,131530,208984,None,208852,240044,None,None,None,None,None,None,None,None,2.2,3.3,3.3,2.15,3.25,3.05,2.1,3.1,3,2,3.2,3.2,None,None,None,2,3.3,3.4,2.2,3.2,3.2,2.2,3.3,3.3,2.15,3.25,3.1,2.2,3.3,2.88


In [25]:
%%sql
DROP VIEW IF EXISTS best_players;

CREATE VIEW best_players AS
WITH pl AS
(
SELECT home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6,
        home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1,
        away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7,
        away_player_8, away_player_9, away_player_10, away_player_11
    FROM Match 
    INNER JOIN best_leagues ON best_leagues.id = Match.league_id 
)

SELECT id, player_api_id, player_name, player_fifa_api_id, date(birthday),  substr(birthday, 1,4) AS birth_year,
        substr(birthday, 6,2) AS birth_month, substr(birthday, 9,2) AS birth_day, height, weight

        FROM player WHERE player_api_id IN (SELECT home_player_1 FROM pl)
                        OR player_api_id IN (SELECT home_player_2 FROM pl)
                        OR player_api_id IN (SELECT home_player_3 FROM pl)
                        OR player_api_id IN (SELECT home_player_4 FROM pl)
                        OR player_api_id IN (SELECT home_player_5 FROM pl)
                        OR player_api_id IN (SELECT home_player_6 FROM pl)
                        OR player_api_id IN (SELECT home_player_7 FROM pl)
                        OR player_api_id IN (SELECT home_player_8 FROM pl)
                        OR player_api_id IN (SELECT home_player_9 FROM pl)
                        OR player_api_id IN (SELECT home_player_10 FROM pl)
                        OR player_api_id IN (SELECT home_player_11 FROM pl)
                        
                        OR player_api_id IN (SELECT away_player_1 FROM pl)
                        OR player_api_id IN (SELECT away_player_2 FROM pl)
                        OR player_api_id IN (SELECT away_player_3 FROM pl)
                        OR player_api_id IN (SELECT away_player_4 FROM pl)
                        OR player_api_id IN (SELECT away_player_5 FROM pl)
                        OR player_api_id IN (SELECT away_player_6 FROM pl)
                        OR player_api_id IN (SELECT away_player_7 FROM pl)
                        OR player_api_id IN (SELECT away_player_8 FROM pl)
                        OR player_api_id IN (SELECT away_player_9 FROM pl)
                        OR player_api_id IN (SELECT away_player_10 FROM pl)
                        OR player_api_id IN (SELECT away_player_11 FROM pl)
                        ;
                        
SELECT COUNT(*) FROM best_players;

 * sqlite:///database.sqlite
Done.
Done.
Done.


COUNT(*)
6102


### Testing the newly created views

**Best_players**

In [26]:
%%sql
SELECT * FROM best_players LIMIT 5;

 * sqlite:///database.sqlite
Done.


id,player_api_id,player_name,player_fifa_api_id,date(birthday),birth_year,birth_month,birth_day,height,weight
9677,2984,Sergio Aragones,111106,1977-02-01,1977,02,01,182.88,176
6856,5440,Marjan Petkovic,122118,1979-05-22,1979,05,22,185.42,185
4784,11316,Jean-Louis Leca,153275,1985-09-21,1985,09,21,180.34,165
1232,11319,Benoit Costil,158121,1987-07-03,1987,07,03,187.96,190
9895,11320,Steeve Elana,111163,1980-07-11,1980,07,11,187.96,187


**Best_teams**

In [27]:
%%sql
SELECT * FROM best_teams LIMIT 5;

 * sqlite:///database.sqlite
Done.


id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
3457,10260,11,Manchester United,MUN
3459,9825,1,Arsenal,ARS
3461,8472,106,Sunderland,SUN
3463,8654,19,West Ham United,WHU
3465,10252,2,Aston Villa,AVL


**Best_leagues**

In [28]:
%%sql
SELECT * FROM best_leagues;

 * sqlite:///database.sqlite
Done.


id,country_id,name
1729,1729,Premier League
4769,4769,League 1
7809,7809,Bundesliga
10257,10257,Serie A
21518,21518,La Liga


## END OF WRANGLING PROCEDURE

As a result of my wrangling analysis, 3 different views were created to diminish the amount of data to the one of my interest for my analysis. New columns such as birth_year, birth_month and birth_day were added to facilitate future calculations and analysis. Name of the leagues were modified to their popular name (how must of people call them).

Major problems such as null data were not identify. From the source I had the information of no null data in this dataset. Structure problems or grammar issues were not visually identified. As the dataset is very long, possible grammar issues in the data may be encountered during the future analysis and will be addressed them. 



# Exploratory Data Analysis

**Exploring teams with the best attributes**

In [33]:
%%sql
SELECT * FROM Team_attributes LIMIT 10;

 * sqlite:///database.sqlite
Done.


id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
1,434,9930,2010-02-22 00:00:00,60,Balanced,None,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
2,434,9930,2014-09-19 00:00:00,52,Balanced,48,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,434,9930,2015-09-10 00:00:00,47,Balanced,41,Normal,54,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
4,77,8485,2010-02-22 00:00:00,70,Fast,None,Little,70,Long,Organised,70,Risky,70,Lots,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
5,77,8485,2011-02-22 00:00:00,47,Balanced,None,Little,52,Mixed,Organised,53,Normal,48,Normal,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
6,77,8485,2012-02-22 00:00:00,58,Balanced,None,Little,62,Mixed,Organised,45,Normal,70,Lots,55,Normal,Organised,40,Medium,40,Press,60,Normal,Cover
7,77,8485,2013-09-20 00:00:00,62,Balanced,None,Little,45,Mixed,Organised,40,Normal,50,Normal,55,Normal,Organised,42,Medium,42,Press,60,Normal,Cover
8,77,8485,2014-09-19 00:00:00,58,Balanced,64,Normal,62,Mixed,Organised,56,Normal,68,Lots,57,Normal,Organised,41,Medium,42,Press,60,Normal,Cover
9,77,8485,2015-09-10 00:00:00,59,Balanced,64,Normal,53,Mixed,Organised,51,Normal,72,Lots,63,Normal,Free Form,49,Medium,45,Press,63,Normal,Cover
10,614,8576,2010-02-22 00:00:00,60,Balanced,None,Little,40,Mixed,Organised,45,Normal,35,Normal,55,Normal,Organised,30,Deep,70,Double,30,Narrow,Offside Trap


## Summarizing Class Attributes Classes

Let's understand better our attributes and classes

I will extract the range of the classes of our attributes

### BuildUpPlaySpeed

In [45]:
%%sql
SELECT buildUpPlaySpeedClass, MIN(buildUpPlaySpeed) AS Min_Points, MAX(buildUpPlaySpeed) AS Max_Points
    FROM team_attributes
    GROUP BY buildUpPlaySpeedClass;


 * sqlite:///database.sqlite
Done.


buildUpPlaySpeedClass,Min_Points,Max_Points
Balanced,34,66
Fast,67,80
Slow,20,33


### BuildUpPlayDribbling

In [44]:
%%sql

SELECT buildUpPlayDribblingClass, MIN(buildUpPlayDribbling) AS Min_Points, MAX(buildUpPlayDribbling) AS Max_Points
    FROM team_attributes
    GROUP BY buildUpPlayDribblingClass;

 * sqlite:///database.sqlite
Done.


buildUpPlayDribblingClass,Min_Points,Max_Points
Little,24,33
Lots,67,77
Normal,34,66


### BuildUpPlayPassing

In [46]:
%%sql

SELECT buildUpPlayPassingClass, MIN(buildUpPlayPassing) AS Min_Points, MAX(buildUpPlayPassing) AS Max_Points
    FROM team_attributes
    GROUP BY buildUpPlayPassingClass;

 * sqlite:///database.sqlite
Done.


buildUpPlayPassingClass,Min_Points,Max_Points
Long,67,80
Mixed,34,66
Short,20,33


### ChanceCreationPassingClass

In [54]:
%%sql

SELECT chanceCreationPassingClass, MIN(chanceCreationPassing) AS Min_Points, MAX(chanceCreationPassing) AS Max_Points
    FROM team_attributes
    GROUP BY chanceCreationPassingClass;

 * sqlite:///database.sqlite
Done.


chanceCreationPassingClass,Min_Points,Max_Points
Normal,34,66
Risky,67,80
Safe,21,33


### ChanceCreationCrossingClass

In [56]:
%%sql

SELECT chanceCreationCrossingClass, MIN(chanceCreationCrossing) AS Min_Points, MAX(chanceCreationCrossing) AS Max_Points
    FROM team_attributes
    GROUP BY chanceCreationCrossingClass;


 * sqlite:///database.sqlite
Done.


chanceCreationCrossingClass,Min_Points,Max_Points
Little,20,33
Lots,67,80
Normal,34,66


### ChanceCreationShootingClass

In [57]:
%%sql

SELECT chanceCreationShootingClass, MIN(chanceCreationShooting) AS Min_Points, MAX(chanceCreationShooting) AS Max_Points
    FROM team_attributes
    GROUP BY chanceCreationShootingClass;

 * sqlite:///database.sqlite
Done.


chanceCreationShootingClass,Min_Points,Max_Points
Little,22,33
Lots,67,80
Normal,34,66


### DefencePressureClass

In [58]:
%%sql

SELECT defencePressureClass, MIN(defencePressure) AS Min_Points, MAX(defencePressure) AS Max_Points
    FROM team_attributes
    GROUP BY defencePressureClass;

 * sqlite:///database.sqlite
Done.


defencePressureClass,Min_Points,Max_Points
Deep,23,33
High,67,72
Medium,34,66


### DefenceAggressionClass

In [59]:
%%sql

SELECT defenceAggressionClass, MIN(defenceAggression) AS Min_Points, MAX(defenceAggression) AS Max_Points
    FROM team_attributes
    GROUP BY defenceAggressionClass;

 * sqlite:///database.sqlite
Done.


defenceAggressionClass,Min_Points,Max_Points
Contain,24,33
Double,67,72
Press,34,66


### DefenceTeamWidthClass

In [60]:
%%sql

SELECT defenceTeamWidthClass, MIN(defenceTeamWidth) AS Min_Points, MAX(defenceTeamWidth) AS Max_Points
    FROM team_attributes
    GROUP BY defenceTeamWidthClass;


 * sqlite:///database.sqlite
Done.


defenceTeamWidthClass,Min_Points,Max_Points
Narrow,29,33
Normal,34,66
Wide,67,73


## Comparing the different Leagues vs Attributes

Before continuing, I will define the summarized attributes according to FIFA in order to understand how they define and are related to the victory or defeat of a team and its style of play. 

**Important Note:** It is fair to say that in football better statistical results not always lead to more victories. Some attributes have more weight in this factor than others. Outside factors such as luck, mistake of players and referees during the game are not taken into account inside this data. So, per safe of this analysis, I will trust the number provided by the attributes in order to define the best teams, at least on these terms. 


**buildUpPlaySpeed:**

Define the speed in which attacks are put together:

| Values | Category | 
| --- | --- |
| 1 - 33 | SLOW |
| 34 - 66 | BALANCED |
| 67 - 100 | FAST |

**buildUpPassing:**

Affects passing distance and support from teammates:

| Values | Category | 
| --- | --- |
| 1 - 33 | SHORT |
| 34 - 66 | MIXED |
| 67 - 100 | LONG |

**buildUpDribbling:**

This parameter defines the creativity of the player in 1 on 1 situations. 

| Values | Category | 
| --- | --- |
| 1 - 33 | LITTLE |
| 34 - 66 | NORMAL |
| 67 - 100 | LOTS |

**ChanceCreationPassingClass:**

Amount of risk in pass decision and run support:

| Values | Category | 
| --- | --- |
| 1 - 33 | SAFE |
| 34 - 66 | NORMAL |
| 67 - 100 | RISKY |

**ChanceCreationCrossingClass:**

The tendency / frequency of crosses into the box

| Values | Category | 
| --- | --- |
| 1 - 33 | LITTLE |
| 34 - 66 | NORMAL |
| 67 - 100 | LOTS |

**ChanceCreationShootingClass:**

The tendency / frequency of shots taken:

| Values | Category | 
| --- | --- |
| 1 - 33 | LITTLE |
| 34 - 66 | NORMAL |
| 67 - 100 | LOTS |

**DefencePressureClass**

Defines how hight he pitch the team will start pressuring:

| Values | Category | 
| --- | --- |
| 1 - 33 | DEEP |
| 34 - 66 | MEDIUM |
| 67 - 100 | HIGH |

**DefenceAggressionClass:**

Defines the team approach to tackling the ball possessor:

| Values | Category | 
| --- | --- |
| 1 - 33 | CONTAIN |
| 34 - 66 | PRESS |
| 67 - 100 | DOUBLE |

**DefenceTeamWidthClass**

Defines how much the team shift to the ball side. The narrower width means that the team tends to cover central position while the wider teams tend to cover more the wings/sides.

| Values | Category | 
| --- | --- |
| 1 - 33 | NARROW |
| 34 - 66 | NORMAL |
| 67 - 100 | WIDE |

[Source](https://www.fifplay.com/fifa-17-tactics/)

## Comparing the different teams per Attribute
### Ranking Leagues according their attributes 

In the next analysis, I will extract the best leagues by parameters through the years to see the variation.

#### Leagues with best buildUpPlaySpeed

In [102]:
%%sql
SELECT l.name AS League, m.Season AS Season,
    
    ROUND(AVG(a.buildUpPlaySpeed),2) AS Avg_PlaySpeed, MAX(a.buildUpPlaySpeed) AS Max_PlaySpeed,
    MIN(a.buildUpPlaySpeed) AS Min_PlaySpeed, 
    
    
        RANK() 
        OVER (PARTITION BY Season ORDER BY ROUND(AVG(a.buildUpPlaySpeed),2) DESC) AS Ranking      
       
        FROM best_leagues l
        INNER JOIN Match m ON m.league_id = l.id
        INNER JOIN Team t ON t.team_api_id = m.home_team_api_id
        INNER JOIN Team_Attributes a ON a.team_api_id = t.team_api_id
  
GROUP BY League, Season;



 * sqlite:///database.sqlite
Done.


League,Season,Avg_PlaySpeed,Max_PlaySpeed,Min_PlaySpeed,Ranking
Premier League,2008/2009,56.77,77,25,1
Bundesliga,2008/2009,56.51,78,31,2
Serie A,2008/2009,56.01,78,26,3
League 1,2008/2009,53.57,70,30,4
La Liga,2008/2009,47.7,71,20,5
Bundesliga,2009/2010,56.39,78,31,1
Premier League,2009/2010,56.12,77,25,2
Serie A,2009/2010,55.3,78,26,3
League 1,2009/2010,53.47,70,30,4
La Liga,2009/2010,47.2,70,20,5


### Creating a top 5 teams per parameter per league

**Top 5 teams with better play Passing score per league**

Ranking the teams with better passing, in other words, those whose player's passes have higher efficacy. Good pass score leads to a player whose passes reaches his teammates

In [143]:
%%sql
WITH stats AS(
SELECT l.name AS League, t.team_long_name AS team_name, ROUND(AVG(a.buildUpPlayPassing),2) AS PlayPassing_Score,
    RANK() 
        OVER (PARTITION BY l.name ORDER BY ROUND(AVG(a.buildUpPlayPassing),2) DESC ) AS Ranking
    FROM best_leagues l
        INNER JOIN Match m ON m.league_id = l.id
        INNER JOIN Team t ON t.team_api_id = m.home_team_api_id
        INNER JOIN Team_Attributes a ON a.team_api_id = t.team_api_id
    GROUP BY League, team_name

)

SELECT * FROM (SELECT * FROM stats WHERE League = 'Bundesliga' LIMIT 5)
UNION ALL
SELECT * FROM (SELECT * FROM stats WHERE League = 'League 1' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM stats WHERE League = 'La Liga' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM stats WHERE League = 'Premier League' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM stats WHERE League = 'Serie A' LIMIT 5 )
;

 * sqlite:///database.sqlite
Done.


League,team_name,PlayPassing_Score,Ranking
Bundesliga,SV Darmstadt 98,77.0,1
Bundesliga,1. FC Köln,61.17,2
Bundesliga,DSC Arminia Bielefeld,59.33,3
Bundesliga,Eintracht Braunschweig,56.0,4
Bundesliga,FC St. Pauli,55.67,5
League 1,OGC Nice,61.83,1
League 1,Grenoble Foot 38,60.0,2
League 1,Toulouse FC,55.33,3
League 1,AJ Auxerre,55.17,4
League 1,Valenciennes FC,54.0,5


**Teams with better builUpDribbling**

Ranking the teams per each league with best one to one players score

In [144]:
%%sql
WITH stats AS(
SELECT l.name AS League, t.team_long_name AS team_name, ROUND(AVG(a.buildUpPlayDribbling),2) AS PlayDribbling_Score,
    RANK() 
        OVER (PARTITION BY l.name ORDER BY ROUND(AVG(a.buildUpPlayDribbling),2) DESC ) AS Ranking
    FROM best_leagues l
        INNER JOIN Match m ON m.league_id = l.id
        INNER JOIN Team t ON t.team_api_id = m.home_team_api_id
        INNER JOIN Team_Attributes a ON a.team_api_id = t.team_api_id
    GROUP BY League, team_name

)

SELECT * FROM (SELECT * FROM stats WHERE League = 'Bundesliga' LIMIT 5)
UNION ALL
SELECT * FROM (SELECT * FROM stats WHERE League = 'League 1' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM stats WHERE League = 'La Liga' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM stats WHERE League = 'Premier League' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM stats WHERE League = 'Serie A' LIMIT 5 )
;

 * sqlite:///database.sqlite
Done.


League,team_name,PlayDribbling_Score,Ranking
Bundesliga,Fortuna Düsseldorf,65.5,1
Bundesliga,SpVgg Greuther Fürth,60.0,2
Bundesliga,1. FC Nürnberg,60.0,2
Bundesliga,Hamburger SV,58.5,4
Bundesliga,VfB Stuttgart,58.0,5
League 1,Olympique de Marseille,69.5,1
League 1,Stade Rennais FC,62.0,2
League 1,FC Sochaux-Montbéliard,62.0,2
League 1,FC Nantes,61.5,4
League 1,SC Bastia,61.0,5


## Comparing the different Teams per Match


In [145]:
%%sql
SELECT * FROM Match LIMIT 6;

 * sqlite:///database.sqlite
Done.


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.73,3.4,5,1.75,3.35,4.2,1.85,3.2,3.5,1.8,3.3,3.75,None,None,None,1.7,3.3,4.33,1.9,3.3,4,1.65,3.4,4.5,1.78,3.25,4,1.73,3.4,4.2
2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.95,3.2,3.6,1.8,3.3,3.95,1.9,3.2,3.5,1.9,3.2,3.5,None,None,None,1.83,3.3,3.6,1.95,3.3,3.8,2,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.6
3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.38,3.3,2.75,2.4,3.3,2.55,2.6,3.1,2.3,2.5,3.2,2.5,None,None,None,2.5,3.25,2.4,2.63,3.3,2.5,2.35,3.25,2.65,2.5,3.2,2.5,2.3,3.2,2.75
4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.44,3.75,7.5,1.4,4,6.8,1.4,3.9,6,1.44,3.6,6.5,None,None,None,1.44,3.75,6,1.44,4,7.5,1.45,3.75,6.5,1.5,3.75,5.5,1.44,3.75,6.5
5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5,3.5,1.65,5,3.5,1.6,4,3.3,1.7,4,3.4,1.72,None,None,

**Creating a view with the new columns of who_lose and who_win to record the number of victories, losses and draws of the different teams**

New columns: 

`who_loss`: team_api_id of the team who lost the game

`who_win`: team_api_id of the tean who won the game  `

In [164]:
%%sql
DROP VIEW IF EXISTS win_loss;

CREATE VIEW win_loss AS
SELECT home_team_api_id, away_team_api_id, league_id,
    CASE WHEN home_team_goal > away_team_goal THEN home_team_api_id
    WHEN away_team_goal > home_team_goal THEN away_team_api_id
    ELSE 'draw'
    END who_won,
    CASE WHEN home_team_goal > away_team_goal THEN away_team_api_id
    WHEN away_team_goal > home_team_goal THEN home_team_api_id
    ELSE 'draw'
    END who_lose
    
    FROM Match;

 * sqlite:///database.sqlite
Done.
Done.


[]



**Creating a view with the ranking of teams according to their victories**

`Columns:` League, Team, Wins, Rank

In [4]:
%%sql
DROP VIEW IF EXISTS victories_ranking;

CREATE VIEW victories_ranking AS
SELECT l.name AS League, t.team_long_name AS Team, COUNT(w.who_won) AS Wins,
RANK()
    OVER(ORDER BY COUNT(w.who_won) DESC) AS ranking
FROM win_loss w
    JOIN best_leagues l ON l.id = w.league_id
    JOIN best_teams t ON t.team_api_id = w.who_won
GROUP BY w.who_won
ORDER BY Wins DESC;

 * sqlite:///database.sqlite
Done.
Done.


[]

### Extracting the teams with more victories

**Top 10 Teams with more victories from the best leagues**

Testing the created view

In [6]:
%%sql
SELECT * FROM victories_ranking LIMIT 10;

 * sqlite:///database.sqlite
Done.


League,Team,Wins,ranking
La Liga,FC Barcelona,35568,1
La Liga,Real Madrid CF,34656,2
Premier League,Manchester United,29184,3
Serie A,Juventus,28539,4
Premier League,Chelsea,26752,5
League 1,Paris Saint-Germain,26600,6
Premier League,Manchester City,26600,6
Bundesliga,FC Bayern Munich,26248,8
Premier League,Arsenal,25840,9
La Liga,Atlético Madrid,25384,10


**Top 10 Teams with more victories from the best leagues**

In [13]:
%%sql

SELECT l.name AS League, t.team_long_name AS Team, COUNT(w.who_lose) AS Loss,
        a.buildUpPlaySpeedClass AS speed_class, a.buildUpPlayDribblingClass AS dribbling_class,
        a.buildUpPlayPassingClass AS passing_class, a.chanceCreationPassingClass AS creation_passing_class,
        a.chanceCreationShootingCLass AS creation_shooting_class, a.defencePressureClass AS pressure_class,
        a.defenceAggressionClass AS agression_class, a.defenceTeamWidthClass AS team_width_class,
        a.defenceDefenderLineClass AS defender_line_class, a.buildUpPlayPositioningClass AS possition_class
FROM win_loss w
    JOIN best_leagues l ON l.id = w.league_id
    JOIN best_teams t ON t.team_api_id = w.who_lose
    JOIN Team_Attributes a ON a.team_api_id = t.team_api_id
GROUP BY w.who_lose
ORDER BY Loss DESC
LIMIT 10;

 * sqlite:///database.sqlite
Done.


League,Team,Loss,speed_class,dribbling_class,passing_class,creation_passing_class,creation_shooting_class,pressure_class,agression_class,team_width_class,defender_line_class,possition_class
La Liga,Getafe CF,126768,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
La Liga,RCD Espanyol,122208,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
Premier League,Sunderland,122208,Balanced,Little,Long,Risky,Normal,Medium,Double,Normal,Cover,Organised
Premier League,Aston Villa,118560,Balanced,Little,Long,Normal,Normal,Medium,Press,Normal,Cover,Organised
Serie A,Chievo Verona,117648,Balanced,Little,Long,Normal,Normal,Deep,Press,Normal,Cover,Organised
Premier League,Stoke City,109440,Balanced,Little,Long,Risky,Normal,Medium,Double,Normal,Cover,Organised
La Liga,Málaga CF,109440,Balanced,Little,Mixed,Normal,Lots,Medium,Press,Normal,Cover,Organised
League 1,OGC Nice,106704,Balanced,Little,Mixed,Normal,Normal,Deep,Press,Normal,Cover,Organised
Serie A,Genoa,106002,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
League 1,FC Lorient,105792,Balanced,Little,Long,Safe,Normal,High,Press,Wide,Offside Trap,Organised


**Top 5 Teams per League with more losses**


In [14]:
%%sql
WITH wins AS (
SELECT l.name AS League, t.team_long_name AS Team, COUNT(w.who_lose) AS Loss,
        a.buildUpPlaySpeedClass AS speed_class, a.buildUpPlayDribblingClass AS dribbling_class,
        a.buildUpPlayPassingClass AS passing_class, a.chanceCreationPassingClass AS creation_passing_class,
        a.chanceCreationShootingCLass AS creation_shooting_class, a.defencePressureClass AS pressure_class,
        a.defenceAggressionClass AS agression_class, a.defenceTeamWidthClass AS team_width_class,
        a.defenceDefenderLineClass AS defender_line_class, a.buildUpPlayPositioningClass AS possition_class
    
FROM win_loss w
    JOIN best_leagues l ON l.id = w.league_id
    JOIN best_teams t ON t.team_api_id = w.who_lose
    JOIN Team_Attributes a ON a.team_api_id = t.team_api_id
GROUP BY w.who_lose
ORDER BY Loss DESC
)

SELECT * FROM (SELECT * FROM wins WHERE League = 'Bundesliga' LIMIT 5)
UNION ALL
SELECT * FROM (SELECT * FROM wins WHERE League = 'League 1' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM wins WHERE League = 'La Liga' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM wins WHERE League = 'Premier League' LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM wins WHERE League = 'Serie A' LIMIT 5 )

 * sqlite:///database.sqlite
Done.


League,Team,Loss,speed_class,dribbling_class,passing_class,creation_passing_class,creation_shooting_class,pressure_class,agression_class,team_width_class,defender_line_class,possition_class
Bundesliga,Hannover 96,101184,Balanced,Little,Long,Normal,Normal,High,Double,Wide,Cover,Organised
Bundesliga,Hamburger SV,92208,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
Bundesliga,VfB Stuttgart,91392,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
Bundesliga,TSG 1899 Hoffenheim,88944,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
Bundesliga,SV Werder Bremen,87312,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Wide,Cover,Organised
League 1,OGC Nice,106704,Balanced,Little,Mixed,Normal,Normal,Deep,Press,Normal,Cover,Organised
League 1,FC Lorient,105792,Balanced,Little,Long,Safe,Normal,High,Press,Wide,Offside Trap,Organised
League 1,Toulouse FC,101232,Balanced,Little,Long,Normal,Normal,Deep,Press,Narrow,Cover,Organised
League 1,Stade Rennais FC,93024,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
League 1,AS Saint-Étienne,92112,Balanced,Little,Mixed,Normal,Normal,Deep,Contain,Narrow,Offside Trap,Organised


**Top 5 Teams Per League with more Wins**


In [15]:
%%sql
WITH wins AS (
SELECT l.name AS League, t.team_long_name AS Team, COUNT(w.who_won) AS Win,
        a.buildUpPlaySpeedClass AS speed_class, a.buildUpPlayDribblingClass AS dribbling_class,
        a.buildUpPlayPassingClass AS passing_class, a.chanceCreationPassingClass AS creation_passing_class,
        a.chanceCreationShootingCLass AS creation_shooting_class, a.defencePressureClass AS pressure_class,
        a.defenceAggressionClass AS agression_class, a.defenceTeamWidthClass AS team_width_class,
        a.defenceDefenderLineClass AS defender_line_class, a.buildUpPlayPositioningClass AS possition_class
FROM win_loss w
    JOIN best_leagues l ON l.id = w.league_id
    JOIN best_teams t ON t.team_api_id = w.who_won
    JOIN Team_Attributes a ON a.team_api_id = t.team_api_id
GROUP BY w.who_won
ORDER BY Win DESC
)

SELECT * FROM (SELECT * FROM wins WHERE League = 'Bundesliga' ORDER BY Win DESC LIMIT 5)
UNION ALL
SELECT * FROM (SELECT * FROM wins WHERE League = 'League 1' ORDER BY Win DESC LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM wins WHERE League = 'La Liga' ORDER BY Win DESC LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM wins WHERE League = 'Premier League' ORDER BY Win DESC LIMIT 5 )
UNION ALL
SELECT * FROM (SELECT * FROM wins WHERE League = 'Serie A' ORDER BY Win DESC LIMIT 5 )

 * sqlite:///database.sqlite
Done.


League,Team,Win,speed_class,dribbling_class,passing_class,creation_passing_class,creation_shooting_class,pressure_class,agression_class,team_width_class,defender_line_class,possition_class
Bundesliga,FC Bayern Munich,157488,Balanced,Little,Mixed,Normal,Lots,High,Press,Normal,Cover,Free Form
Bundesliga,Borussia Dortmund,128112,Fast,Little,Mixed,Normal,Lots,Medium,Double,Normal,Cover,Organised
Bundesliga,Bayer 04 Leverkusen,111792,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
Bundesliga,FC Schalke 04,103632,Balanced,Little,Mixed,Normal,Lots,High,Press,Wide,Cover,Organised
Bundesliga,VfL Wolfsburg,95472,Balanced,Little,Mixed,Normal,Lots,Medium,Press,Normal,Cover,Organised
League 1,Paris Saint-Germain,159600,Balanced,Little,Mixed,Normal,Normal,High,Double,Wide,Offside Trap,Organised
League 1,Olympique Lyonnais,139536,Balanced,Little,Mixed,Normal,Lots,Medium,Press,Normal,Cover,Organised
League 1,LOSC Lille,134064,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised
League 1,Olympique de Marseille,130416,Balanced,Little,Mixed,Normal,Lots,Medium,Press,Normal,Cover,Organised
League 1,Girondins de Bordeaux,114912,Balanced,Little,Mixed,Normal,Normal,Medium,Press,Normal,Cover,Organised


## Note: Pending summary from the last 2 queries!!!


# Pending:
* Create function for the above to analyze all the parameters and rank the leagues by season  --> **Done (couldnt create function)**

* Do the same for the best teams of the leagues --> Top 5 teams by league per season  -->**Done**


* See the teams of more victories per league and year with top players  --> **Done**


* Teams with more scored goals and against goals (add their rank)

* Check other attributes of teams (add their rank)

* Compare the teams with more victories and losses and their style of game  --> **Done with the addition of attributes to to the top 5 teams**

* See evolution of teams through the year (thing about it)

* Best players 

* Mean of players age per team. Oldest and youngest teams